In [1]:
import keras
import pandas as pd
import numpy as np
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.initializers import RandomUniform

%matplotlib inline

Using TensorFlow backend.
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [33]:
boneData = pd.read_csv('boneage-training-dataset.csv')
paths = []
print(len(boneData['id']))
boneData = boneData[(boneData.boneage <= 4.*12) | (boneData.boneage >= 15.5*12)]
for i in boneData['id']:
    paths.append('boneage-training-dataset/' + str(i) + '.png')
print(len(paths))

12611
1240


In [19]:
X = []
for path in paths:
    img = image.load_img(path, target_size=(500,500), grayscale=True)
    img = image.img_to_array(img)
    X.append(img)

/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [32]:
Y = np.array(boneData['boneage'])
#print(Y.max())
#print(np.median(Y))
#print(Y.min())

print(Y[0:10])

pos = 0
for i in range(0,len(Y)):
    if(Y[i]/12)< 11: 
        Y[i] = 0
    else: 
        Y[i] = 1
        pos += 1
        
print(Y[0:10])
print(float(pos/len(Y)))
split = float(pos/len(Y))
Y = to_categorical(Y)

[ 12  36 188   4  36  24  30  33  24  32]
[0 0 1 0 0 0 0 0 0 0]
0.5419354838709678


In [5]:
yTrain = Y[:1000]
yVal = Y[1000:]

In [20]:
xTrain = np.array(X)

In [21]:
xTrain = xTrain/255.
xTrain = xTrain.astype(np.float32)

In [22]:
xVal = xTrain[1000:]
xTrain = xTrain[:1000]

In [24]:
x = xTrain

In [34]:
kmodel = Sequential([
    Conv2D(16, (10,10), input_shape=(500, 500, 1), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(32, (3, 3), input_shape=(500, 500, 1), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Flatten(input_shape=(200, 200, 1)),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(2, activation = 'softmax')
])
print("built")
kmodel.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print("compiled")
fittedModel = kmodel.fit(xTrain, yTrain, validation_data=[xVal,yVal], epochs = 5)
print("trained")

built
compiled
Train on 1000 samples, validate on 240 samples
Epoch 1/5
1000/1000 [==============================] - 489s 489ms/step - loss: 7.4541 - acc: 0.5350 - val_loss: 0.6921 - val_acc: 0.5583
Epoch 2/5
1000/1000 [==============================] - 487s 487ms/step - loss: 7.4541 - acc: 0.5350 - val_loss: 0.6921 - val_acc: 0.5583
Epoch 3/5
1000/1000 [==============================] - 493s 493ms/step - loss: 7.4541 - acc: 0.5350 - val_loss: 0.6921 - val_acc: 0.5583
Epoch 4/5
1000/1000 [==============================] - 544s 544ms/step - loss: 7.4541 - acc: 0.5350 - val_loss: 0.6921 - val_acc: 0.5583
Epoch 5/5
1000/1000 [==============================] - 578s 578ms/step - loss: 7.4541 - acc: 0.5350 - val_loss: 0.6921 - val_acc: 0.5583
trained


In [35]:
sgd = keras.optimizers.SGD(lr = .01)
kmodel.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics=['accuracy'], )

In [ ]:
h = kmodel.fit(xTrain, yTrain, epochs = 10, validation_data=[xVal, yVal], verbose = 1)

Train on 1000 samples, validate on 240 samples
Epoch 1/10
1000/1000 [==============================] - 479s 479ms/step - loss: 7.4949 - acc: 0.5350 - val_loss: 0.6921 - val_acc: 0.5583
Epoch 2/10
1000/1000 [==============================] - 518s 518ms/step - loss: 7.4949 - acc: 0.5350 - val_loss: 0.6921 - val_acc: 0.5583
Epoch 3/10
 704/1000 [====================>.........] - ETA: 2:02 - loss: 7.4638 - acc: 0.5369

In [ ]:
print(h.history['acc'])
print(split)